In [20]:
import os
import pandas as pd

In [21]:
def loopSeparateAccuracyRegardingDays(predictionPath):
    ''' 
    loop to each prediccted csv
    assign each data to dict and return
    trait: [[list of accuracy], [], ...]
    '''

    # loop into each csv file
    targetDfDict = {}
    for predictionFile in os.listdir(predictionPath):
        # check if the file is nused file: start with "."
        if(predictionFile[0] == '.'):
            continue

        # create file path
        predictionFilePath = os.path.join(predictionPath, predictionFile)

        # read the csv file
        df = pd.read_csv(predictionFilePath)

        target = predictionFile.split('_')[0]
        if(target not in [*targetDfDict]):
            targetDfDict[target] = []
        targetDfDict[target].append(df)


    return targetDfDict

def calculateAverageAccuracyOfTraits(targetDfDict):
    ''' 
    loop into the dict of list(model) of list(result)
    and combine all model result
    then calculate the average result
    '''
    returnDict = {}

    for eachTarget in [*targetDfDict]:

        # Concatenate all the df of current target
        combined_df = pd.concat(targetDfDict[eachTarget], ignore_index=True)

        # Group by date and compute the mean of r2_score and avg_predicted_std
        avg_df = combined_df.groupby("date").agg({
            "r2_score": "mean",
            "avg_predicted_std": "mean"
        }).reset_index()

        returnDict[eachTarget] = avg_df

    return returnDict

def saveAverageResultDict(averageResultDict, savePath = "./growthPredictions/averageResultByDays.csv"):
    # Assuming your dictionary is called `results_dict`
    combined_rows = []

    for target_name, df in averageResultDict.items():
        temp_df = df.copy()
        temp_df.insert(0, "target", target_name)  # Insert 'target' as the first column
        combined_rows.append(temp_df)

    # Concatenate all into a single DataFrame
    final_df = pd.concat(combined_rows, ignore_index=True)

    # Save to CSV
    final_df.to_csv(savePath, index=False)


In [ ]:
resultByDatePath = "./growthPredictions/resultByDate2025"
targetDfDict = loopSeparateAccuracyRegardingDays(resultByDatePath)

In [ ]:
averageResultDict = calculateAverageAccuracyOfTraits(targetDfDict)
saveAverageResultDict(averageResultDict, savePath = "./growthPredictions/averageResultByDays2025.csv")